In [1]:
%matplotlib inline

# Example plot for LFPy: Mainen and Sejnowski (1996) spike waveforms
This is an example scripts using LFPy with an active cell model adapted from
Mainen and Sejnowski, Nature 1996, for the original files, see
http://senselab.med.yale.edu/modeldb/ShowModel.asp?model=2488

This scripts is set up to use the model, where the active conductances are set
in the file "active_declarations_example2.hoc", and uses the mechanisms from
the .mod-files provided here. For this example to work, run "nrnivmodl" in
this folder to compile these mechanisms
(i.e. /$PATHTONEURON/nrn/x86_64/bin/nrnivmodl).

A single excitatory synapse drive the neuron into producing a single action-
potential, and the local field potential are calculated on a dense 2D-grid
on around the soma.


Copyright (C) 2017 Computational Neuroscience Group, NMBU.

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

In [2]:
import os
from os.path import join
import sys
if sys.version < '3':
    from urllib2 import urlopen
else:    
    from urllib.request import urlopen
import zipfile
import ssl
from warnings import warn
import LFPy
import neuron
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PolyCollection
import scipy.io as sio
import csv

## Fetch Mainen&Sejnowski 1996 model files:

In [3]:
if not os.path.isfile(join('cells', 'cells', 'j4a.hoc')):
    # get the model files:
    u = urlopen('http://senselab.med.yale.edu/ModelDB/eavBinDown.asp?o=2488&a=23&mime=application/zip',
                context=ssl._create_unverified_context())
    localFile = open('patdemo.zip', 'wb')
    localFile.write(u.read())
    localFile.close()
    #unzip:
    myzip = zipfile.ZipFile('patdemo.zip', 'r')
    myzip.extractall('.')
    myzip.close()

# compile mod files every time, because of incompatibility with Hay2011 files:
if "win32" in sys.platform:
    pth = "cells"
    warn("no autompile of NMODL (.mod) files on Windows. " 
         + "Run mknrndll from NEURON bash in the folder cells and rerun example script")
    if not pth in neuron.nrn_dll_loaded:
        neuron.h.nrn_load_dll(pth+"/nrnmech.dll")
    neuron.nrn_dll_loaded.append(pth)
else:
    os.system('''
              cd cells
              nrnivmodl
              ''')
    neuron.load_mechanisms('cells')

## Simulation parameters:
Define parameters, using dictionaries. It is possible to set a few more parameters for each class or functions, but we chose to show only the most important ones here.

In [4]:
# define cell parameters used as input to cell-class
cellParameters = {
    'morphology' : 'morphologies/L5_Mainen96_wAxon_LFPy.hoc',
    'cm' : 1.0,                 # membrane capacitance
    'Ra' : 150,                 # axial resistance
    'v_init' : -65,             # initial crossmembrane potential
    'passive' : True,           # turn on passive mechanism
    'passive_parameters' : {'g_pas' : 1./30000, 'e_pas' : -65}, # passive params
    'nsegs_method' : 'lambda_f',# method for setting number of segments,
    'lambda_f' : 500,           # segments are isopotential at this frequency
    'dt' : 2**-5,               # dt of LFP and NEURON simulation.
    'tstart' : 0.0,             # start time, recorders start at t=0
    'tstop' : 20.0,               # stop time of simulation
    'custom_code'  : ['active_declarations_example2.hoc'], # will run this file
}

# Synaptic parameters, corresponding to a NetCon synapse built into NEURON
synapseParameters = {
    'idx' : 0,               # insert synapse on index "0", the soma
    'e' : 0.,                # reversal potential of synapse
    'syntype' : 'Exp2Syn',   # conductance based double-exponential synapse
    'tau1' : 1.0,            # Time constant, rise
    'tau2' : 1.0,            # Time constant, decay
    'weight' : 0.03,         # Synaptic weight
    'record_current' : True, # Will enable synapse current recording
}

np.random.seed(1676)
shift_x=np.random.uniform(-150.,150.,10)

np.random.seed(14)
shift_y=np.random.uniform(-25.,25.,10)

np.random.seed(251)
shift_z=np.random.uniform(-150.,150.,10)

#np.random.seed(511)
#randParameters=np.random.choice(cellParameters,size=10)

colors=[]
np.random.seed(562)
for i in range(10):
    colors.append(tuple(np.random.rand(3))+(0.5,))
    
#print(shift_x)
#print(shift_y)
#print(shift_z)
#print(randParameters)
#print(colors)
#for x, y, z in zip(shift_x, shift_y, shift_z):
#    print(x, y, z)

# Generate the grid in xz-plane over which we calculate local field potentials
lattice_constance = 10.0
u_0 = (lattice_constance, 0.0)
u_1 = (0.5*lattice_constance, 0.5*np.sqrt(3.0)*lattice_constance)
id=0
x=np.zeros(10*10*3)
y=np.zeros(10*10*3)
z=np.zeros(10*10*3)
for j in range(-1, 2):
    for k in range(-3, 4):
        for i in range(-5, 6):
            #print(i,j,k)
            x_test=u_0[0]*i+u_1[0]*k
            if(x_test>-155.0 and x_test<155.0):
                x[id]=x_test
                y[id]=lattice_constance*j
                z[id]=u_0[1]*i+u_1[1]*k
                id += 1          
            else:
                #print("truncated")
                pass

# define parameters for extracellular recording electrode, using optional method
electrodeParameters = {
    'sigma' : 0.3,              # extracellular conductivity
    'x' : x,        # x,y,z-coordinates of contact points
    'y' : y,
    'z' : z,
    'method' : 'soma_as_point',  #treat soma segment as sphere source
}

In [5]:
# Generate the grid in xz-plane over which we calculate local field potentials
x, y, z = np.mgrid[-10:10, -10:10, -10:10] * 10

# define parameters for extracellular recording electrode, using optional method
electrodeParameters_grid = {
    'sigma' : 0.3,              # extracellular conductivity
    'x' : x.flatten(),        # x,y,z-coordinates of contact points
    'y' : y.flatten(),
    'z' : z.flatten(),
    'method' : 'soma_as_point',  #treat soma segment as sphere source
}

## Main simulation procedure, setting up extracellular electrode, cell, synapse:

In [6]:
# create extracellular electrode object
electrode = LFPy.RecExtElectrode(**electrodeParameters_grid)

# Initialize cell instance, using the LFPy.Cell class
cell = LFPy.Cell(**cellParameters)
# set the position of midpoint in soma to Origo (not needed, this is the default)
cell.set_pos(x = 0, y = 0, z = 0)
# rotate the morphology 90 degrees around z-axis
cell.set_rotation(z = np.pi/2)

# attach synapse with parameters and set spike time
synapse = LFPy.Synapse(cell, **synapseParameters)
synapse.set_spike_times(np.array([1.0, 15.0]))

# perform NEURON simulation, results saved as attributes in the cell instance
cell.simulate(electrode = electrode)

TypeError: ufunc 'true_divide' output (typecode 'd') could not be coerced to provided output parameter (typecode 'l') according to the casting rule ''same_kind''

## Plotting of simulation results:

In [ ]:
from example_suppl import plot_ex2
fig = plot_ex2(cell, electrode)
# Optional: saving the figure
# fig.savefig('LFPy-example-6.pdf', dpi=300)

In [ ]:
fig = plt.figure(dpi=300)

ax = fig.add_axes([0.0,0.0,0.7,1.0], frameon=True)

#plot electrode
ax.plot(electrode.x, electrode.z,'.', marker='^', color='g')

i = 0
limLFP = abs(electrode.LFP).max()
for LFP in electrode.LFP:
    tvec = cell.tvec*0.2 + electrode.x[i] + 2
    if abs(LFP).max() >= 1:
        factor = 2
        color='r'
    elif abs(LFP).max() < 0.25:
        factor = 50
        color = 'b'
    else:
        factor = 10
        color = 'g'
    trace = LFP*factor + electrode.z[i]
    ax.plot(tvec, trace, color=color, lw = 1)
    i += 1

#plot cell morphology
zips = []
for x, z in cell.get_idx_polygons():
    zips.append(list(zip(x, z)))

polycol = PolyCollection(zips,
                         edgecolors='none',
                         facecolors=colors[0], alpha=0.4)
ax.add_collection(polycol)

#ax.plot([100, 200], [-400, -400], 'k', lw=1, clip_on=False)
#ax.text(150, -470, r'100$\mu$m', va='center', ha='center')

ax.axis('on')
ax.axis([-200., 200., -200., 200.])

#plot Synapse
ax.plot(cell.xmid[cell.synidx],cell.zmid[cell.synidx], 'o', ms=5,
        markeredgecolor='none',
        markerfacecolor='r')

ax = fig.add_axes([0.75, 0.0, 0.25, 1.0])
i = 0
limLFP = abs(electrode.LFP).max()
for LFP in electrode.LFP:
    tvec = cell.tvec
    if abs(LFP).max() >= 1:
        factor = 2
        color='r'
    elif abs(LFP).max() < 0.25:
        factor = 50
        color = 'b'
    else:
        factor = 10
        color = 'g'
    trace = LFP*factor + i
    ax.plot(cell.tvec, trace, color=color, lw = 0.25)
    i += 1

In [ ]:
import plotly.graph_objects as go
import scipy.io as sio

In [ ]:
print(electrode.LFP.shape)
print(electrode.LFP[:,201])

In [ ]:
fig = go.Figure(data=go.Volume(
    x=x.flatten(),
    y=z.flatten(),
    z=y.flatten(),
    value= np.log10(electrode.LFP[:,0]),
    isomin=np.amin(np.log10(electrode.LFP)),
    isomax=np.amax(np.log10(electrode.LFP)),
    opacity=0.1, # needs to be small to see through all surfaces
    surface_count=25, # needs to be a large number for good volume rendering
    ))
fig.update_layout(scene_xaxis_showticklabels=False,
                  scene_yaxis_showticklabels=False,
                  scene_zaxis_showticklabels=False)
fig.show(renderer="notebook")